In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    
    
    def value_DP_iteration(V, env, T, reward_type = "gamma_discount", gamma = 1.0):
        print("----- T = {} ------".format(T))
        assert (T >= 1)
        Q = np.ndarray([env.nS, env.nA])
        print(Q.shape)
        for s in range(env.nS):
            #Q(x, a) { \sum_{x in X} P_{x->x_prime | a} * (1/T R_{x->x_prime | a} + (T-1) / T * V(x_prime) }
            for a in range(env.nA):
                trans = env.P[s][a][0] #确定性的，s a唯一决定 s_prime, prob = 1.0
                s_prime = trans[1]
                R = trans[2]
                prob = trans[0]
                assert prob == 1.0
                if reward_type == "gamma_discount":
                    Q_x_a = R + gamma * V[s_prime]
                elif reward_type == "T":
                    Q_x_a = ( 1.0 / T ) * R + (T-1.0) / T * V[s_prime]
                else:
                    raise ValueError('不支持的策略评估函数')
                Q[s, a] = Q_x_a
            
        print("Q = ", V)

        return Q
        
        
        
                
                
        
    

    
    T = 1
    #nS 为 状态的个数 在这里是格子的个数
    V = np.zeros(env.nS)
    while True:
        
        Q = value_DP_iteration(V, env, T,reward_type = "gamma_discount", gamma = 1.0)
        V_prime = np.max(Q, axis = 1)
        V_diff = np.abs(V_prime - V)
        diff_max = np.max(V_diff)
        if diff_max <= theta:
            break
            
        V = V_prime
        T += 1

    
    # policy，由于是确定性的策略，似乎没必要是个矩阵？
    policy = np.zeros([env.nS, env.nA])
    Q = value_DP_iteration(V, env, T,reward_type = "gamma_discount", gamma = 1.0 )
    
    best_action = np.argmax(Q, axis = 1)

    
    for s in range(env.nS):
        policy[s, best_action[s]] = 1.0
        
    
    # Implement!
    return policy, V

In [4]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

----- T = 1 ------
(16, 4)
Q =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
----- T = 2 ------
(16, 4)
Q =  [ 0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0.]
----- T = 3 ------
(16, 4)
Q =  [ 0. -1. -2. -2. -1. -2. -2. -2. -2. -2. -2. -1. -2. -2. -1.  0.]
----- T = 4 ------
(16, 4)
Q =  [ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]
----- T = 4 ------
(16, 4)
Q =  [ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]
Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]


In [5]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)